## Imports

In [ ]:
import pandas as pd
import numpy as np

import keras

from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras

## Loading Datasets

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Urdu/urdu_train.csv')
df_val = pd.read_csv("/content/drive/MyDrive/Urdu/urdu_val.csv")

In [ ]:
df_train_Tweet = pd.DataFrame(df_train.text)
df_val_Tweet = pd.DataFrame(df_val.text)
df_train_Label = pd.DataFrame(df_train.label)
df_val_Label = pd.DataFrame(df_val.label)


## Preprocessing

In [ ]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))

def preprocessing(document):
        document = regex.sub(' ', document)
        document = re.sub(r'[0-9]', '', document)
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)             # remove all single characters
        document = re.sub(r'\s+', ' ', document, flags=re.I)            # Substituting multiple spaces with single space
        return document
    


In [ ]:
corpus_train = df_train_Tweet.text.apply(preprocessing)
corpus_val = df_val_Tweet.text.apply(preprocessing)

## TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,5))
cv = TfidfVectorizer(max_features=15000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_val).toarray()
X_train.shape

(1038, 15000)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Label Encoding

In [ ]:
Y_train = np.ravel(df_train_Label)
Y_test = np.ravel(df_val_Label)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y_train)
Y_train = le.transform(Y_train)
Y_test = le.transform(Y_test)

#Model Creation and Validation

## SIMPLE ANN

In [ ]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(512, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(256, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               7680512   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 7,844,865
Trainable params: 7,844,865
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_save_path='/content/m1'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=40,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=30,
                    callbacks=callbacks)

Epoch 1/30
35/35 [==============================] - 1s 15ms/step - loss: 0.6913 - accuracy: 0.5539 - val_loss: 0.6875 - val_accuracy: 0.5802
Epoch 2/30
35/35 [==============================] - 0s 8ms/step - loss: 0.6714 - accuracy: 0.6002 - val_loss: 0.6816 - val_accuracy: 0.5725
Epoch 3/30
35/35 [==============================] - 0s 8ms/step - loss: 0.6499 - accuracy: 0.5992 - val_loss: 0.6751 - val_accuracy: 0.5725
Epoch 4/30
35/35 [==============================] - 0s 9ms/step - loss: 0.6227 - accuracy: 0.6233 - val_loss: 0.6679 - val_accuracy: 0.5725
Epoch 5/30
35/35 [==============================] - 0s 9ms/step - loss: 0.5857 - accuracy: 0.7052 - val_loss: 0.6582 - val_accuracy: 0.5725
Epoch 6/30
35/35 [==============================] - 0s 8ms/step - loss: 0.5352 - accuracy: 0.8044 - val_loss: 0.6480 - val_accuracy: 0.5840
Epoch 7/30
35/35 [==============================] - 0s 9ms/step - loss: 0.4670 - accuracy: 0.9133 - val_loss: 0.6338 - val_accuracy: 0.5954
Epoch 8/30
35/35 [=

In [ ]:
model.load_weights(model_save_path)
import keras
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.92      0.49      0.64       112
           R       0.72      0.97      0.82       150

    accuracy                           0.76       262
   macro avg       0.82      0.73      0.73       262
weighted avg       0.80      0.76      0.75       262



###Creating padded sequences

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(corpus_train)

X_train = tokenizer.texts_to_sequences(corpus_train)
X_test = tokenizer.texts_to_sequences(corpus_val)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(corpus_train[2])
print(len(X_train[2]))

 لندن سپورٹس ڈیسک ای سی بی کے نیشنل سلیکٹر ایڈ سمتھ کا کہنا ہے کہ جانی بیئرسٹو اور معین علی سمیت کسی کھلاڑی پر ٹیم کے دروازے بند نہیں کئے گئے بلکہ ویسٹ انڈیز کیخلاف پہلے ٹیسٹ کیلئے منتخب شدہ سکواڈ اس بات کا عکس ہے کہ انہیں سفید گیند کے میچز میں قابل غور سمجھا جائیگا۔ انہوں نے تصدیق کی کہ آنیوالے دنوں میں وائٹ بال میچز کے دوران معین علی اور جانی بیئرسٹو کو ایکشن میں دیکھا جا سکے گا ۔ ایڈ سمتھ کا کہنا تھا کہ اگر دونوں کھلاڑی ٹیسٹ سکواڈ سے باہر ہوئے تو یہ اس اعتبار سے ایک اچھی بات ہے کہ انہیں وائٹ بال میچز کے دوران پورا موقع دیا جائیگا۔ 
117


In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 512

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## ANN with Embedding matrix

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 100)          2049700   
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               26214912  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 28,428,965
Trainable params: 28,428,965
Non-trainable params: 0
__________________________________________

In [ ]:
model_save_path='/content/m2'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=15,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
33/33 [==============================] - 2s 28ms/step - loss: 0.6854 - accuracy: 0.5694 - val_loss: 0.6768 - val_accuracy: 0.5496
Epoch 2/30
33/33 [==============================] - 1s 23ms/step - loss: 0.5766 - accuracy: 0.6272 - val_loss: 0.7356 - val_accuracy: 0.5649
Epoch 3/30
33/33 [==============================] - 1s 21ms/step - loss: 0.3819 - accuracy: 0.8536 - val_loss: 0.7853 - val_accuracy: 0.5458
Epoch 4/30
33/33 [==============================] - 1s 21ms/step - loss: 0.0691 - accuracy: 0.9913 - val_loss: 0.8325 - val_accuracy: 0.5687
Epoch 5/30
33/33 [==============================] - 1s 23ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.9625 - val_accuracy: 0.5840
Epoch 6/30
33/33 [==============================] - 1s 21ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.9908 - val_accuracy: 0.5763
Epoch 7/30
33/33 [==============================] - 1s 21ms/step - loss: 6.3869e-04 - accuracy: 1.0000 - val_loss: 0.9975 - val_accuracy: 0.5687
Epoch 8/3

In [ ]:
model.load_weights(model_save_path)
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.52      0.30      0.38       112
           R       0.60      0.79      0.69       150

    accuracy                           0.58       262
   macro avg       0.56      0.55      0.54       262
weighted avg       0.57      0.58      0.56       262



####Reduced dimension to 50

In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 50)           1024850   
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 1,025,371
Trainable params: 1,025,371
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_save_path='/content/m3'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=25,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
33/33 [==============================] - 1s 14ms/step - loss: 0.6890 - accuracy: 0.5443 - val_loss: 0.6834 - val_accuracy: 0.5725
Epoch 2/30
33/33 [==============================] - 0s 8ms/step - loss: 0.6767 - accuracy: 0.5780 - val_loss: 0.6776 - val_accuracy: 0.5725
Epoch 3/30
33/33 [==============================] - 0s 8ms/step - loss: 0.6659 - accuracy: 0.5780 - val_loss: 0.6718 - val_accuracy: 0.5725
Epoch 4/30
33/33 [==============================] - 0s 8ms/step - loss: 0.6510 - accuracy: 0.5780 - val_loss: 0.6627 - val_accuracy: 0.5725
Epoch 5/30
33/33 [==============================] - 0s 8ms/step - loss: 0.6275 - accuracy: 0.5838 - val_loss: 0.6495 - val_accuracy: 0.5802
Epoch 6/30
33/33 [==============================] - 0s 8ms/step - loss: 0.5909 - accuracy: 0.6879 - val_loss: 0.6308 - val_accuracy: 0.6221
Epoch 7/30
33/33 [==============================] - 0s 8ms/step - loss: 0.5400 - accuracy: 0.8266 - val_loss: 0.6079 - val_accuracy: 0.6832
Epoch 8/30
33/33 [=

In [ ]:
model.load_weights(model_save_path)
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.82      0.55      0.66       112
           R       0.73      0.91      0.81       150

    accuracy                           0.76       262
   macro avg       0.77      0.73      0.73       262
weighted avg       0.77      0.76      0.75       262



##CNN

In [ ]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 100)          2049700   
_________________________________________________________________
conv1d (Conv1D)              (None, 512, 64)           19264     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [ ]:
model_save_path='/content/m4'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
33/33 [==============================] - 30s 21ms/step - loss: 0.6884 - accuracy: 0.5530 - val_loss: 0.6819 - val_accuracy: 0.5725
Epoch 2/30
33/33 [==============================] - 0s 12ms/step - loss: 0.6778 - accuracy: 0.5780 - val_loss: 0.6811 - val_accuracy: 0.5725
Epoch 3/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6709 - accuracy: 0.5780 - val_loss: 0.6802 - val_accuracy: 0.5725
Epoch 4/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6615 - accuracy: 0.5780 - val_loss: 0.6781 - val_accuracy: 0.5725
Epoch 5/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6362 - accuracy: 0.5925 - val_loss: 0.6692 - val_accuracy: 0.5725
Epoch 6/30
33/33 [==============================] - 0s 11ms/step - loss: 0.5862 - accuracy: 0.7341 - val_loss: 0.6599 - val_accuracy: 0.5840
Epoch 7/30
33/33 [==============================] - 0s 11ms/step - loss: 0.5227 - accuracy: 0.7832 - val_loss: 0.6478 - val_accuracy: 0.6260
Epoch 8/30
3

In [ ]:
model.load_weights(model_save_path)
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.75      0.61      0.67       112
           R       0.74      0.85      0.79       150

    accuracy                           0.74       262
   macro avg       0.74      0.73      0.73       262
weighted avg       0.74      0.74      0.74       262



In [ ]:
embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 512, 50)           1024850   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 64)           9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [ ]:
model_save_path='/content/m5'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/30
33/33 [==============================] - 1s 18ms/step - loss: 0.6889 - accuracy: 0.5424 - val_loss: 0.6838 - val_accuracy: 0.5725
Epoch 2/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6798 - accuracy: 0.5780 - val_loss: 0.6815 - val_accuracy: 0.5725
Epoch 3/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6746 - accuracy: 0.5780 - val_loss: 0.6808 - val_accuracy: 0.5725
Epoch 4/30
33/33 [==============================] - 0s 10ms/step - loss: 0.6686 - accuracy: 0.5780 - val_loss: 0.6793 - val_accuracy: 0.5725
Epoch 5/30
33/33 [==============================] - 0s 9ms/step - loss: 0.6631 - accuracy: 0.5780 - val_loss: 0.6767 - val_accuracy: 0.5725
Epoch 6/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6483 - accuracy: 0.5780 - val_loss: 0.6733 - val_accuracy: 0.5725
Epoch 7/30
33/33 [==============================] - 0s 11ms/step - loss: 0.6250 - accuracy: 0.5886 - val_loss: 0.6682 - val_accuracy: 0.5725
Epoch 8/30
33/

In [ ]:
model.load_weights(model_save_path)
import keras
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.70      0.57      0.63       112
           R       0.72      0.82      0.77       150

    accuracy                           0.71       262
   macro avg       0.71      0.70      0.70       262
weighted avg       0.71      0.71      0.71       262



## BILSTM

In [ ]:
embedding_dim = 100
max_len = 512

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 512, 100)          2049700   
_________________________________________________________________
bidirectional (Bidirectional (None, 512, 512)          731136    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [ ]:
model_save_path='/content/m6'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=50,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks = callbacks)

Epoch 1/50
33/33 [==============================] - 15s 311ms/step - loss: 0.6808 - accuracy: 0.5780 - val_loss: 0.6837 - val_accuracy: 0.5725
Epoch 2/50
33/33 [==============================] - 9s 286ms/step - loss: 0.6770 - accuracy: 0.5780 - val_loss: 0.6833 - val_accuracy: 0.5725
Epoch 3/50
33/33 [==============================] - 9s 287ms/step - loss: 0.6706 - accuracy: 0.5780 - val_loss: 0.6823 - val_accuracy: 0.5725
Epoch 4/50
33/33 [==============================] - 9s 287ms/step - loss: 0.6663 - accuracy: 0.5780 - val_loss: 0.6815 - val_accuracy: 0.5725
Epoch 5/50
33/33 [==============================] - 9s 288ms/step - loss: 0.6522 - accuracy: 0.5780 - val_loss: 0.6824 - val_accuracy: 0.5725
Epoch 6/50
33/33 [==============================] - 9s 288ms/step - loss: 0.6401 - accuracy: 0.5780 - val_loss: 0.6886 - val_accuracy: 0.5725
Epoch 7/50
33/33 [==============================] - 9s 289ms/step - loss: 0.6287 - accuracy: 0.5780 - val_loss: 0.6898 - val_accuracy: 0.5725
Epoch

In [ ]:
model.load_weights(model_save_path)

class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.71      0.57      0.63       112
           R       0.72      0.83      0.77       150

    accuracy                           0.72       262
   macro avg       0.72      0.70      0.70       262
weighted avg       0.72      0.72      0.71       262



####Reduced dimension to 50

In [ ]:
embedding_dim = 50
max_len = 120

model = Sequential()

model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)))

model.add(layers.GlobalMaxPooling1D())
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(2, activation='softmax'))

model.add(layers.Dense(1, activation='sigmoid'))

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 512, 50)           1024850   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512, 512)          628736    
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 20)                10260     
_________________________________________________________________
dropout_5 (Dropout)          (None, 20)               

In [ ]:
model_save_path='/content/m7'

callbacks = [keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)]

In [ ]:
history = model.fit(X_train, Y_train,
                     epochs=30,
                     verbose=1,
                     validation_data=(X_test, Y_test),
                     batch_size=32,
                    callbacks=callbacks)

Epoch 1/100
33/33 [==============================] - 14s 308ms/step - loss: 0.6769 - accuracy: 0.5751 - val_loss: 0.6844 - val_accuracy: 0.5725
Epoch 2/100
33/33 [==============================] - 9s 283ms/step - loss: 0.6537 - accuracy: 0.6551 - val_loss: 0.6820 - val_accuracy: 0.5725
Epoch 3/100
33/33 [==============================] - 9s 283ms/step - loss: 0.6182 - accuracy: 0.7303 - val_loss: 0.6790 - val_accuracy: 0.5725
Epoch 4/100
33/33 [==============================] - 9s 284ms/step - loss: 0.6005 - accuracy: 0.7351 - val_loss: 0.7318 - val_accuracy: 0.5725
Epoch 5/100
33/33 [==============================] - 9s 284ms/step - loss: 0.5536 - accuracy: 0.7977 - val_loss: 0.7284 - val_accuracy: 0.5725
Epoch 6/100
33/33 [==============================] - 9s 283ms/step - loss: 0.4868 - accuracy: 0.8362 - val_loss: 0.8164 - val_accuracy: 0.4275
Epoch 7/100
33/33 [==============================] - 9s 284ms/step - loss: 0.4913 - accuracy: 0.8266 - val_loss: 0.7883 - val_accuracy: 0.427

KeyboardInterrupt: ignored

In [ ]:
model.load_weights(model_save_path)
import keras
class_names = df_train.label.unique()
y_pred = model.predict(X_test)>0.5
y_pred = [str(x[0]).replace("False","0").replace("True","1") for x in y_pred]
y_pred = [int(x) for x in y_pred]

y_pred = le.inverse_transform(y_pred)
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.81      0.41      0.54       112
           R       0.68      0.93      0.78       150

    accuracy                           0.71       262
   macro avg       0.74      0.67      0.66       262
weighted avg       0.73      0.71      0.68       262

